In [ ]:
import numpy as np
import pandas as pd
import talib as ta #安装talib用这三行代码：/usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)" 或 /bin/zsh -c "$(curl -fsSL https://gitee.com/cunkai/HomebrewCN/raw/master/Homebrew.sh)" ; brew install ta-lib ; pip install ta-lib
import tushare as ts
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.rcParams['axes.unicode_minus']=False #确保可以显示‘-’号
mpl.rcParams['font.sans-serif'] = ['SimHei'] #确保中文显示正常

stock = ts.get_k_data('600030', '2016-06-01', '2017-06-30')
stock.sort_index(inplace=True)

stock['cci'] = ta.CCI(np.asarray(stock['high']), np.asarray(stock['low']), np.asarray(stock['close']), timeperiod=20) #用最高价、最低价、收盘价计算CCI，取时间周期为20。查用法可以用ta.CCI?

#绘制cci指标图
plt.subplot(2, 1, 1)
plt.title('600030 CCI指标图')
plt.gca().axes.get_xaxis().set_visible(False) #不显示x轴
stock['close'].plot(figsize = (10,8))
plt.legend()
plt.subplot(2, 1, 2)
stock['cci'].plot(figsize = (10,8))
plt.legend()
plt.show()

stock['yes_cci'] = stock['cci'].shift(1) #生成昨日的CCI
stock['daybeforeyes_cci'] = stock['cci'].shift(2) #生成前天的CCI

stock['signal'] = np.where(np.logical_and(stock['daybeforeyes_cci']<-100, stock['yes_cci']>-100), 1, np.nan) #np.logical_and()等价于&
#stock['signal'] = np.where((stock['daybeforeyes_cci']<-100) & (stock['yes_cci']>-100), 1, np.nan)
stock['signal'] = np.where(np.logical_and(stock['daybeforeyes_cci']>100, stock['yes_cci']<100),-1, stock['signal'])
stock['signal'] = stock['signal'].fillna(method='ffill')
stock['signal'] = stock['signal'].fillna(0)

plt.subplot(3, 1, 1)
plt.title('600030 CCI开仓图')
plt.gca().axes.get_xaxis().set_visible(False)
stock['close'].plot(figsize = (10,8))
plt.legend(loc='upper left')
plt.subplot(3, 1, 2)
stock['cci'].plot(figsize = (10,8))
plt.legend(loc='upper left')
plt.gca().axes.get_xaxis().set_visible(False)
plt.subplot(3, 1, 3)
stock['signal'].plot(figsize = (10,8),marker='o',linestyle='')
plt.legend(loc='upper left')
plt.show()

stock['pct_change'] = stock['close'].pct_change() #股票每日收益率
stock['strategy_return'] = stock['pct_change'] * stock['position'] #策略每日收益率，注意这里不能加.shift(1)，因为策略的条件是用昨天的CCI和前天的CCI测的，不需要今天的CCI，所以今天是可以知道并进行买卖的
stock['return'] = (stock['pct_change']+1).cumprod() #股票累积收益率
stock['strategy_cum_return'] = (1 + stock['strategy_return']).cumprod() #计算策略累积收益率

stock[['return', 'strategy_cum_return']].plot(figsize = (12,6)) #将股票累积收益率和策略累积收益率绘图
plt.title('600030 CCI收益图')
plt.legend(loc='upper left')
plt.show()